<a href="https://colab.research.google.com/github/WOLFOX1717/WOLFOX1717/blob/main/darkmode_pdf_convert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pymupdf pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 51.3 MB/s eta 0:00:00


In [11]:
pip install pymupdf pillow

import fitz  # PyMuPDF
from PIL import Image, ImageOps
import os

def convert_pdf_to_dark_mode(input_pdf_path, output_pdf_path, bg_color=(0, 0, 0), fg_color=(255, 255, 255), dpi=150, image_quality=80):
    """
    Converts a PDF to dark mode by inverting the colors.

    Args:
        input_pdf_path (str): Path to the input PDF.
        output_pdf_path (str): Path to save the output PDF.
        bg_color (tuple): Background color in RGB (default is black).
        fg_color (tuple): Foreground color in RGB (default is white).
        dpi (int): Resolution of the output images in DPI (default is 150).
        image_quality (int): Quality of JPEG images (1-100, default is 80).
    """
    # Open the PDF
    pdf_document = fitz.open(input_pdf_path)

    # Create a temporary directory to save processed images
    temp_images_dir = "temp_images"
    os.makedirs(temp_images_dir, exist_ok=True)

    # Process each page
    image_paths = []
    for page_number in range(len(pdf_document)):
        # Get the page as a pixmap (image) with reduced DPI
        page = pdf_document.load_page(page_number)
        pix = page.get_pixmap(dpi=dpi)

        # Convert pixmap to PIL Image
        image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

        # Invert colors and apply dark mode
        inverted_image = ImageOps.invert(image)
        dark_mode_image = ImageOps.colorize(inverted_image.convert("L"), black=bg_color, white=fg_color)

        # Save the image as a compressed JPEG
        image_path = os.path.join(temp_images_dir, f"page_{page_number + 1}.jpg")
        dark_mode_image.save(image_path, "JPEG", quality=image_quality)
        image_paths.append(image_path)

    # Create a new PDF with the processed images
    pdf_writer = fitz.open()
    for image_path in image_paths:
        # Insert each processed image as a page
        img = Image.open(image_path)
        rect = fitz.Rect(0, 0, img.width, img.height)
        pdf_page = pdf_writer.new_page(width=img.width, height=img.height)
        pdf_page.insert_image(rect, filename=image_path)

    # Save the final PDF with compression
    pdf_writer.save(output_pdf_path, deflate=True, compression_effort=9)
    pdf_writer.close()

    # Clean up temporary images
    for image_path in image_paths:
        os.remove(image_path)
    os.rmdir(temp_images_dir)

    print(f"Dark mode PDF saved to {output_pdf_path}")

# Example usage
convert_pdf_to_dark_mode(
    "/content/TELEDETECCION Y CAMBIO GLOBAR COMPENDIO.pdf",
    "/content/sample_data/Output",
    bg_color=(0, 0, 0),
    fg_color=(255, 255, 255),
    dpi=150,               # Lower resolution to reduce size
    image_quality=80       # Adjust image quality for compression
)


Dark mode PDF saved to /content/sample_data/Output
